In [1]:
import torch

import torch.nn as nn

import transformers


class FakeRoBERTModel(nn.Module):
    def __init__(self):
        super(FakeRoBERTModel, self).__init__()
        # embedding
        self.bert = transformers.AutoModel.from_pretrained(
            "dumitrescustefan/bert-base-romanian-cased-v1"
        )

        # first convolutional layers
        self.conv1_2 = nn.Conv1d(768, 512, kernel_size=1, stride=1, padding=0)
        self.drop1_3 = nn.Dropout(p=0.2)
        self.pool1_4 = nn.MaxPool1d(kernel_size=1, stride=2, padding=0)

        # second convolutional layers
        self.conv1_5 = nn.Conv1d(512, 256, kernel_size=1, stride=1, padding=0)
        self.drop1_5 = nn.Dropout(p=0.2)
        self.pool1_5 = nn.MaxPool1d(kernel_size=1, stride=2, padding=0)
        self.conv1_6 = nn.Conv1d(256, 64, kernel_size=1, stride=1, padding=0)
        self.drop1_6 = nn.Dropout(p=0.2)
        self.pool1_6 = nn.MaxPool1d(kernel_size=1, stride=2, padding=0)

        # flatten
        self.flatt_7 = nn.Flatten(1, -1)

        # dense layers
        self.dense1_8 = nn.Linear(64, 32)
        self.dense1_9 = nn.Linear(32, 1)

        self.dense_1 = nn.Linear(768, 256)
        self.dense_3 = nn.Linear(256, 32)
        self.dense_4 = nn.Linear(32, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False,
        )

        # x, y = pooled_output.size()
        # pooled_output = torch.reshape(pooled_output, (x, y, 1))

        # conv1_2 = self.pool1_4(self.drop1_3(torch.relu(self.conv1_2(pooled_output))))
        # x = conv1_2
        # x = self.pool1_5(self.drop1_5(F.relu(self.conv1_5(x))))
        # x = self.pool1_6(self.drop1_6(F.relu(self.conv1_6(x))))
        # x = self.flatt_7(x)
        # x = self.dense1_8(x)

        x = self.drop1_6(pooled_output)
        x = self.dense_1(x)
        x = self.dense_3(x)
        x = self.dense_4(x)
        x = torch.sigmoid(x)

        return x


In [2]:
model = FakeRoBERTModel()

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
BERT_PATH = "dumitrescustefan/bert-base-romanian-cased-v1"
tokenizer = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
device = torch.device("cpu")

In [4]:
with torch.no_grad():
    max_len: int = 512
    encoded_text = tokenizer.encode_plus(
        "Salutare, lume!",
        add_special_tokens=True,
        max_length=max_len,
        padding="longest",
        return_token_type_ids=True,
        return_attention_mask=True,
        truncation=True,
    )

    ids = encoded_text["input_ids"]
    mask = encoded_text["attention_mask"]
    token_type_ids = encoded_text["token_type_ids"]

    padding_length = max_len - len(ids)
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)

    ids = torch.tensor([ids], dtype=torch.long)
    mask = torch.tensor([mask], dtype=torch.long)
    token_type_ids = torch.tensor([token_type_ids], dtype=torch.long)

    ids = ids.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)

    yhat = model(input_ids=ids, attention_mask=mask, token_type_ids=token_type_ids)


In [5]:
from torchviz import make_dot

make_dot(yhat[0], params=dict(list(model.named_parameters()))).render("FakeRoBERT", format="png")


'FakeRoBERT.png'

In [6]:
import hiddenlayer as hl

graph = hl.build_graph(model, args=(ids, mask, token_type_ids), transforms=[])
graph.theme = hl.graph.THEMES["blue"].copy()
graph.save("FakeRoBERT_")


/home/ioachimlihor/Licenta/Project/licenta/licenta-env/lib64/python3.10/site-packages/torch/onnx/symbolic_helper.py:324: UserWarning: Type cannot be inferred, which might cause exported graph to produce incorrect results.
  warnings.warn("Type cannot be inferred, which might cause exported graph to produce incorrect results.")
/home/ioachimlihor/Licenta/Project/licenta/licenta-env/lib64/python3.10/site-packages/torch/onnx/symbolic_opset9.py:1672: UserWarning: Dropout is a training op and should not be exported in inference mode. For inference, make sure to call eval() on the model and to export it with param training=False.
  warnings.warn("Dropout is a training op and should not be exported in inference mode. "


In [7]:
input_names = ['Sentence']
output_names = ['yhat']
torch.onnx.export(
    model,
    (ids, mask, token_type_ids),
    "rnn.onnx",
    input_names=input_names,
    output_names=output_names,
)


In [8]:
import onnx

# Load the ONNX model
model = onnx.load("rnn.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %Sentence[INT64, 1x512]
  %attention_mask[INT64, 1x512]
  %input.3[INT64, 1x512]
) initializers (
  %bert.embeddings.word_embeddings.weight[FLOAT, 50000x768]
  %bert.embeddings.position_embeddings.weight[FLOAT, 512x768]
  %bert.embeddings.token_type_embeddings.weight[FLOAT, 2x768]
  %bert.embeddings.LayerNorm.weight[FLOAT, 768]
  %bert.embeddings.LayerNorm.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.query.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.key.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.value.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.dense.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.LayerNorm.weight[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.LayerNorm.bias[FLOAT, 768]
  %bert.encoder.layer.0.intermediate.dense.bias[FLOAT, 3072]
  %bert.encoder.layer.0.output.dense.bias[FLOAT, 768]
  %bert.encoder.layer.0.output.LayerNorm.weight[FLOAT, 768]
  %bert.encoder.layer.0.output.Layer

In [9]:
import onnxruntime as ort

ort_session = ort.InferenceSession("alexnet.onnx")

outputs = ort_session.run(
    None,
    {"actual_input_1": np.random.randn(10, 3, 224, 224).astype(np.float32)},
)
print(outputs[0])

ModuleNotFoundError: No module named 'onnxruntime'